In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pypsa

In [64]:
run = "nresults_full_3H_ws"
prefix = "/mnt/c/Users/scl38887/Documents/git/aldehyde/workflow/subworkflows/pypsa-earth-sec/results/" + run + "/postnetworks/"
output = "/mnt/c/Users/scl38887/Documents/git/aldehyde/results/" + run + "/graphics/rldc/"

In [65]:
path = "elec_s_10_ec_lc1.0_Co2L0.50_3H_2030_0.15_DF_20export.nc"
n = pypsa.Network(prefix + path)

INFO:pypsa.io:Imported network elec_s_10_ec_lc1.0_Co2L0.50_3H_2030_0.15_DF_20export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


All nodes

In [82]:
capex = n.links[n.links.carrier == "H2 Electrolysis"].p_nom_opt * n.links[n.links.carrier == "H2 Electrolysis"].capital_cost
opex = n.links_t.p0.loc[:, n.links.carrier == "H2 Electrolysis"].sum() * n.links[n.links.carrier == "H2 Electrolysis"].marginal_cost

In [83]:
el_cost = n.links_t.p0.loc[:, n.links.carrier == "H2 Electrolysis"].multiply(n.buses_t.marginal_price.loc[:, n.buses.carrier == "AC"].values, axis = 0).sum()

In [84]:
hydrogen = (-1) * n.links_t.p1.loc[:, n.links.carrier == "H2 Electrolysis"].sum()

In [120]:
cost_comp = pd.concat([capex, opex, el_cost, hydrogen], axis=1).reset_index(drop=True)
cost_comp.columns = ["capex", "opex", "el_cost", "hydrogen"]
cost_comp.index = hydrogen.index
cost_comp["total_cost"] = cost_comp["capex"] + cost_comp["opex"] + cost_comp["el_cost"]
cost_comp["capex_share"] = cost_comp["capex"] / cost_comp["total_cost"]
cost_comp["lcoh"] = cost_comp["total_cost"] / cost_comp["hydrogen"]

In [123]:
cost_comp["capex_share"].mean()

0.4350203199618859

In [125]:
cost_comp["capex"].mean() / cost_comp["total_cost"].mean()

0.46424525835148284

In [113]:
cost_comp.head()

,capex,opex,el_cost,lcoh,total_cost,capex_share
0,9.725323e+03,1.577507,1.167520e+04,183.058269,2.140210e+04,0.454410
1,3.410962e+04,6.509403,3.983611e+04,180.298396,7.395224e+04,0.461239
2,1.485295e+05,29.882242,2.158162e+05,173.472255,3.643757e+05,0.407628
3,4.095840e+04,7.736209,4.932272e+04,176.957933,9.028886e+04,0.453637
4,6.021060e+06,1673.149295,1.193394e+07,170.000862,1.795667e+07,0.335311


Plotting

Marginal price

In [12]:

n.buses_t.marginal_price.loc[:, n.buses.carrier == "H2"].iloc[:, 0] #.plot() # .describe() #).sum()

snapshot
2013-01-01 00:00:00    154.953504
2013-01-01 03:00:00    154.953493
2013-01-01 06:00:00    154.953481
2013-01-01 09:00:00    154.942126
2013-01-01 12:00:00    154.942110
                          ...    
2013-12-31 09:00:00    154.951874
2013-12-31 12:00:00    154.951877
2013-12-31 15:00:00    154.951921
2013-12-31 18:00:00    154.953514
2013-12-31 21:00:00    154.953515
Name: H2 export bus, Length: 2920, dtype: float64